## Clusters de potagers en Île-de-France

- L'objectif est de représenter les regroupements de potagers de Île-de-France.
- Pour cela, on va utiliser le plugin **MarkerCluster** de la librairie **Folium**.

In [1]:
# import de quelques packages utiles

import os
import json
import folium
import pandas as pd
import geopandas
import numpy as np
from folium import plugins
from folium.plugins import MarkerCluster

In [2]:
# instanciation d'un carte

maCarte = folium.Map(
            location=[48.8333, 2.5033],
            tiles = 'cartodbpositron',
            zoom_start = 8
)

In [3]:
# affichage de la carte

#maCarte

In [4]:
# on crée un dataframe avec les potagers

centroides = geopandas.read_file("./data/centro_WGS84.geojson")

In [5]:
# on check le DF

centroides.head()

,ID_Jard,Jard_Surf,Zone,Dpt_Code,Com_Code,long,lat,longgg,lattt,geometry
0,1447,340.85,Roissy,95,95580,2.5652,49.092,2.565225,49.091613,POINT (2.56522 49.09161)
1,1448,107.66,Roissy,95,95580,2.5732,49.083,2.573197,49.082602,POINT (2.57320 49.08260)
2,1449,44.63,Roissy,95,95580,2.5633,49.090,2.563272,49.090147,POINT (2.56327 49.09015)
3,1450,101.08,Roissy,95,95580,2.5660,49.091,2.565987,49.090778,POINT (2.56599 49.09078)
4,1451,84.78,Roissy,95,95580,2.5669,49.092,2.566934,49.091832,POINT (2.56693 49.09183)


In [6]:
# on crée un nouveau tableau des positions avec numpy

tableauPositions = np.array([
    centroides['lattt'],
    centroides['longgg'],
    centroides['ID_Jard']
])

In [7]:
tableauPositions

array([[4.90916126e+01, 4.90826017e+01, 4.90901475e+01, ...,
        4.87352062e+01, 4.87321522e+01, 4.87321695e+01],
       [2.56522451e+00, 2.57319685e+00, 2.56327156e+00, ...,
        2.20252894e+00, 2.20648551e+00, 2.20659725e+00],
       [1.44700000e+03, 1.44800000e+03, 1.44900000e+03, ...,
        7.72100000e+03, 7.76500000e+03, 7.76600000e+03]])

In [8]:
# transposition du tableau

tableauPositionsT = tableauPositions.T

In [9]:
# dimensions du tableau transposé

tableauPositionsT.shape

(8098, 3)

In [10]:
# instanciation d'une variable pour stocker une fonction javascript (écrite comme un string)
# modifiera le css de base
# --> ceci dans le but d'avoir des icones de carottes à la place des icones de base.

icon_create_function = """
    function(cluster) {
    var childCount = cluster.getChildCount(); 
    var c = ' marker-cluster-';

    if (childCount < 50) {
        c += 'large';
    } else if (childCount < 300) {
        c += 'medium';
    } else {
        c += 'small';
    }

    return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) });
    }
    """

In [11]:
# on fait appel au MarkerCluster avec la variable créée avant 

mc = MarkerCluster(name='Potagers', icon_create_function=icon_create_function)

In [12]:
# itération pour tous les potagers

for i in range(8098):
    carrot_icon_link = "./img/carrot_1.png"
    carrot_icon = folium.features.CustomIcon(carrot_icon_link, icon_size=(30, 29))
    folium.Marker([tableauPositionsT[i][0], tableauPositionsT[i][1]], 
                  icon=carrot_icon, 
                  popup= 'Potager '+str(int(tableauPositionsT[i][2]))).add_to(mc)
    mc.add_to(maCarte)

In [13]:
# ajout du plugin Fullscreen

plugins.Fullscreen(
        position = 'topright',
        title = 'Plein écran',
        force_separate_button = True).add_to(maCarte)

In [14]:
# ajout d'un fond de carte Stamen Toner

coucheST = folium.TileLayer('Stamen Toner')
coucheST.add_to(maCarte)

In [15]:
# ajout d'un fond de carte OpenStreetMap

coucheOSM = folium.TileLayer('openstreetmap')
coucheOSM.add_to(maCarte)

In [16]:
# ajout plugin LayerControl

folium.LayerControl().add_to(maCarte)

In [17]:
#maCarte

In [18]:
# on sauvegarde la carte
maCarte.save('carteClustersPotagers.html')

In [19]:
###